In [12]:
import os
import requests
import json
import pandas as pd
import hvplot.pandas
import sqlalchemy
from sqlalchemy import inspect
from dotenv import load_dotenv
import numpy as np

In [13]:
load_dotenv()

True

In [14]:
database_connection_string = os.getenv("DATABASE_URL")
database_schema = os.getenv("DATABASE_SCHEMA")

display(type(database_connection_string))
display(type(database_schema))

str

str

In [15]:
engine = sqlalchemy.create_engine(database_connection_string)
inspector = inspect(engine)
inspector.get_table_names()

['token_attribute',
 'network',
 'collection',
 'contract_map',
 'token',
 'whale',
 'api',
 'trade',
 'social_media',
 'data_analysis']

In [24]:
os_top_collection_index = """
SELECT t.contract_id,
       c.name,
       DATE_TRUNC('day', t.timestamp) as year_day_month,
       SUM(t.volume) as total_volume,
       SUM(t.num_trades) as total_num_trades,
       SUM(t.unique_buyers) as total_unique_buyers
FROM trade t
INNER JOIN collection c ON c.contract_id = t.contract_id
INNER JOIN network n ON n.network_id = c.network_id
WHERE n.network_id = 'ethereum' 
AND c.name NOT IN ('','New 0x495f947276749Ce646f68AC8c248420045cb7b5eLock', 'pieceofshit', 'Uniswap V3 Positions NFT-V1','More Loot',
                  'NFTfi Promissory Note','dementorstownwtf','ShitBeast','mcgoblintownwtf','LonelyPop','Pablos','For the Culture','Hype Pass', 'Moonbirds Oddities', 'AIMoonbirds', 'Bound NFT CloneX')
GROUP BY t.contract_id, c.name, DATE_TRUNC('day', t.timestamp)
HAVING MAX(avg_price) > 0
ORDER BY DATE_TRUNC('day', t.timestamp)  ASC
"""

In [25]:
os_top_collection_index_df = pd.read_sql_query(os_top_collection_index, con=engine)
os_top_collection_index_df

,contract_id,name,year_day_month,total_volume,total_num_trades,total_unique_buyers
0,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-23,3,20,8
1,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-24,4,20,7
2,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-25,2,13,7
3,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-26,2,18,8
4,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-27,8,35,8
...,...,...,...,...,...,...
14613,ethereum:b852c6b5892256c264cc2c888ea462189154d8d7,Rektguy,2022-08-06,1,6,6
14614,ethereum:659a4bdaaacc62d2bd9cb18225d9c89b5b697a5a,We Are All Going to Die,2022-08-06,10,6,6
14615,ethereum:ed5af388653567af2f388e6224dc7c4b3241c544,Azuki,2022-08-06,54,6,6
14616,ethereum:60e4d786628fea6478f785a6d7e704777c86a7c6,MutantApeYachtClub,2022-08-06,241,9,9


In [26]:
# filter the query for only the top ten collections listed on OpenSea"
os_top_collection_index_df = os_top_collection_index_df[os_top_collection_index_df['name'].str.contains('CryptoPunks|BoredApeYachtClub|MutantApeYachtClub|Otherdeed|Azuki|CloneX|Moonbirds|Doodles|Cool Cats|BoredApeKennelClub')==True]

os_top_collection_index_df

,contract_id,name,year_day_month,total_volume,total_num_trades,total_unique_buyers
0,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-23,3,20,8
1,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-24,4,20,7
2,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-25,2,13,7
3,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-26,2,18,8
4,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-27,8,35,8
...,...,...,...,...,...,...
14584,ethereum:49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,CloneX,2022-08-06,124,14,14
14586,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2022-08-06,1149,11,7
14598,ethereum:ba30e5f9bb24caa003e9f2f0497ad287fdf95623,BoredApeKennelClub,2022-08-06,93,7,7
14615,ethereum:ed5af388653567af2f388e6224dc7c4b3241c544,Azuki,2022-08-06,54,6,6


In [27]:
os_top_collection_index_df = os_top_collection_index_df[['year_day_month', 'total_volume', 'total_num_trades', 'total_unique_buyers']]

In [28]:
os_top_collection_index_df.columns = ['date', 'volume', 'num_trades', 'unique_buyers']

In [29]:
os_top_collection_index_df

,date,volume,num_trades,unique_buyers
0,2017-06-23,3,20,8
1,2017-06-24,4,20,7
2,2017-06-25,2,13,7
3,2017-06-26,2,18,8
4,2017-06-27,8,35,8
...,...,...,...,...
14584,2022-08-06,124,14,14
14586,2022-08-06,1149,11,7
14598,2022-08-06,93,7,7
14615,2022-08-06,54,6,6


In [30]:
os_top_collection_index_df = os_top_collection_index_df[os_top_collection_index_df['date'] > '2020-12-31']

In [31]:
os_top_collection_index_volume = os_top_collection_index_df.groupby('date')['volume'].sum()
os_top_collection_index_num_trades = os_top_collection_index_df.groupby('date')['num_trades'].sum()
os_top_collection_index_unique_buyers = os_top_collection_index_df.groupby('date')['unique_buyers'].sum()

In [32]:
os_top_collection_index_unique_buyers

date
2021-01-01      3
2021-01-02      4
2021-01-03      5
2021-01-04     11
2021-01-05     15
             ... 
2022-08-02     58
2022-08-03     70
2022-08-04    108
2022-08-05    176
2022-08-06    133
Name: unique_buyers, Length: 583, dtype: int64

# OPTION ONE

In [35]:
overlay_1 = os_top_collection_index_volume.hvplot(
    height=700, 
    width=1000, 
    yformatter='%.0f', 
    ylabel = 'Volume in ETH', 
    xlabel = 'Date',
    title = 'Open Sea Volume for Top Collections January 2021 to August 2022',
    color = 'green',
    label = 'Volume in ETH'
)

In [37]:
overlay_1

:Curve   [date]   (volume)

# OPTION 2

In [38]:
overlay_2 = os_top_collection_index_num_trades.hvplot(
    height=700, 
    width=1000, 
    yformatter='%.0f', 
    ylabel = 'Number of Trades', 
    xlabel = 'Date',
    title = 'Open Sea Number of Trades for Top Collections January 2021 to August 2022',
    color = 'blue',
    label = 'Number of Trades'
)

In [39]:
overlay_2

:Curve   [date]   (num_trades)

# OPTION 3

In [40]:
overlay_3 = os_top_collection_index_unique_buyers.hvplot(
    height=700, 
    width=1000, 
    yformatter='%.0f', 
    xlabel = 'Date',
    ylabel = 'Nmber of Unique Buyers',
    title = 'Open Sea Unique Buyers for Top Collections January 2021 to August 2022',
    color = 'red',
    label = 'Number of Unique Buyers',
)

In [41]:
overlay_3

:Curve   [date]   (unique_buyers)

# OPTION 4

In [42]:
overlay = overlay_1 * overlay_2 * overlay_3

In [49]:
overlay.opts(
    title = 'Open Sea Top Collections January 2021 to August 2022',
    show_legend = True,
    legend_position = 'top_left'
)

:Overlay
   .Curve.Volume_in_ETH           :Curve   [date]   (volume)
   .Curve.Number_of_Trades        :Curve   [date]   (num_trades)
   .Curve.Number_of_Unique_Buyers :Curve   [date]   (unique_buyers)

# OPTION 5

In [45]:
overlay_2_and_3 = (overlay_2 * overlay_3)
overlay_2_and_3.opts(
    title = 'Open Sea Number of Trades and Unique Buyers January 2021 to August 2022',
    show_legend = True,
    legend_position = 'top_left'
)

:Overlay
   .Curve.Number_of_Trades        :Curve   [date]   (num_trades)
   .Curve.Number_of_Unique_Buyers :Curve   [date]   (unique_buyers)

In [46]:
side_by_side = overlay_2_and_3 + overlay_1

In [47]:
side_by_side

:Layout
   .Overlay.I           :Overlay
      .Curve.Number_of_Trades        :Curve   [date]   (num_trades)
      .Curve.Number_of_Unique_Buyers :Curve   [date]   (unique_buyers)
   .Curve.Volume_in_ETH :Curve   [date]   (volume)